In [478]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
import re, sys, random, glob, json, unidecode, unicodedata
import ipdb, time, sys, os, pickle
from IPython.core.display import display, HTML
from IPython.display import clear_output
from urllib.request import urlopen
from collections import Counter
from bs4 import BeautifulSoup  

display(HTML("<style>.container { width:100% !important; }</style>"))

# Matplotlib Configuration
pylab.rcParams.update({'legend.fontsize': 'medium',
                       'font.size'      : 12,
                       'figure.figsize' : (10, 10),
                       'axes.labelsize' : 'medium',
                       'axes.titlesize' : 'medium',
                       'axes.grid'      : 'on',
                       'xtick.labelsize': 'medium',
                       'ytick.labelsize': 'medium'})

def print_runtime(start: float, printer: bool = True) -> None:
    end = time.time()
    if printer:
        print(f'Runtime: {int((end - start) // 60)} min {int((end - start) % 60):2d} sec')
    else:
        return f'({int((end - start) // 60)} min {int((end - start) % 60):2d} sec)'

/var/folders/jf/qykqz6ks0vs56cb_zyf_7x_00000gn/T/ipykernel_1248/597161407.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


---
# Extract names from emails

In [655]:
from openai import OpenAI
client = OpenAI()

def compute_sequence_prob(response, alpha=0.7):
    logprob = []
    for item in response.choices[0].logprobs.content:
        #print(item.token, (item.logprob))
        logprob.append(item.logprob)

    avg_logprob = sum(logprob) / len(logprob) ** .7
    return np.exp(avg_logprob)

def get_prompt(first_name, email, bio):
    prompt = f"""
    First Name: {first_name}
    Email: {email}
    Bio: {bio}
 """
    return prompt

def get_response(model, sys_message, prompt):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": sys_message},
            {"role": "user", "content": prompt}
        ],
        temperature=0.01,
        logprobs=True,
        top_logprobs=5
    )
    
    prob = compute_sequence_prob(response)
    
    return response, prob


In [656]:
fname = 'First Name Change - Interexo 1_29_24.xlsx - Data - No Change.csv'
df = pd.read_csv(fname)
df.insert(1, 'likelihood', [0]*len(df))
df.rename({'Given Name': 'Preferred Name'}, axis=1, inplace=True)
df = df.astype({'Preferred Name':str, 'likelihood':float})
df.head()

,Preferred Name,likelihood,First Name,Last Name,Email,Company Name,Bio,Address,Country,State,City,Zip Code,Phone #,Phone #.1,Website,Profile Urls
0,nan,0.0,Christopher,Cheung,chris.cheung@fbxbrokers.com,Florida Business Exchange,Chris Cheung is a \t\t\t\t\t\tTampa business b...,550 Reo St.\t\t\t\t\t\t\t\t\t\t\t\t\tSuite 300,United States,FL,Tampa,33602,813-785-9089,NaN,https://fbxbrokers.com,https://brokers.interexo.com/united-states_flo...
1,nan,0.0,William,Gaines,bill@franpro.com,Franprocom,William Gaines is a business broker of the we...,Bowie,United States,MD,20715,NaN,443-583-3070,NaN,https://franpro.com,https://brokers.interexo.com/united-states_mar...
2,nan,0.0,Anthony,Shard,tony@shardmb.com,Shard Marketing,Anthony Shard is a \t\t\t\t\t\tCortez business...,12325 Baypointe Ter,United States,FL,Cortez,34215-2566,810-577-5291,NaN,https://shardmb.com,https://brokers.interexo.com/united-states_flo...
3,nan,0.0,Antonio,Bardy,tonybardy@gmail.com,Florida Connexion Properties,Antonio Bardy is a \t\t\t\t\t\tWindermere busi...,1961 Westhill Run,United States,FL,Windermere,34786-6230,407-535-3639,NaN,NaN,https://brokers.interexo.com/united-states_flo...
4,nan,0.0,Elizabeth,Berger,liz@lizberger.com,Berger Realty Group,Elizabeth Berger is a Coral Springs business b...,1955 University Dr,United States,FL,Coral Springs,33071,954-344-8222,954-650-7848,https://lizberger.com/,https://brokers.interexo.com/united-states_flo...


In [ ]:
start = time.time()
model = "gpt-3.5-turbo"
# model = "gpt-4-0125-preview"

sys_message = f"""Given the values for "First Name", "Email" and "Bio" of a company employee, your job is to predict what his/her Preferred Name is. For instance, a person with the first name "Anthony" might prefer being called "Tony", and the evidence of it could be that his email is "tony_pearson@gmail.com" or in his Bio he might be referred to as Tony instead of Anthony. 
However, if you have no evidence for his "Preferred Name" being different than his "First Name" in the email or the "Bio", then you must have no doubts about outputting the original "First Name" and you must output his "First Name" as is. 
If the email features a common preferred version of a First Name, then you can confidently output the preferred version featured in the email. 
Provide your response as a single word and nothing else. Do not include any other text, or any explanation or any special formatting like using backticks (``). Don't make up a full sentence. DO NOT, I repeat, DO NOT start your response with the word Preferred.

I'm providing the "First Name", "Email", and the "Bio" of a business professional below."""

p = 7 # 46, 50
for idx, row in df.iterrows():
    first_name, email, bio = row['First Name'], row['Email'], row['Bio']
    prompt = get_prompt(first_name, email, bio)
    response, prob = get_response(model, sys_message, prompt)
    content = response.choices[0].message.dict()['content']
    
    if prob <= 0.95:
        print(f"{idx:3d}   first_name:{first_name:12s}-> {content:8s}  prob:{round(prob,5)}   {'✔️' if first_name != content else ''}")
        print("-> prob <= 0.80. Re-trying with gpt-4-0125-preview")
        response, prob = get_response("gpt-4-0125-preview", sys_message, prompt)
        
        content = response.choices[0].message.dict()['content']
    df.loc[idx, 'Preferred Name'] = content
    df.loc[idx, 'likelihood'] = round(prob, 2)
    print(f"{idx:3d}   first_name:{first_name:12s}-> {content:8s}  prob:{round(prob,5)}   {'✔️' if first_name != content else ''}")
    
    compute_sequence_prob(response)


  0   first_name:Christopher -> Chris     prob:0.99722   ✔️
  1   first_name:William     -> Bill      prob:0.99898   ✔️
  2   first_name:Anthony     -> Tony      prob:0.99855   ✔️
  3   first_name:Antonio     -> Tony      prob:0.99893   ✔️
  4   first_name:Elizabeth   -> Liz       prob:0.99397   ✔️
  5   first_name:Eric        -> Eric      prob:0.98598   
  6   first_name:Edward      -> Edward    prob:0.95172   
  7   first_name:Andrea      -> Andrea    prob:0.72929   
-> prob <= 0.80. Re-trying with gpt-4-0125-preview
  7   first_name:Andrea      -> Andrea    prob:1.0   
  8   first_name:Adam        -> Adam      prob:0.99349   
  9   first_name:Allan       -> Allan     prob:0.98398   
 10   first_name:Anita       -> Anita     prob:0.97907   
 11   first_name:Alex        -> Alex      prob:0.96879   
 12   first_name:Andres      -> Andres    prob:0.82872   
-> prob <= 0.80. Re-trying with gpt-4-0125-preview
 12   first_name:Andres      -> Andres    prob:1.0   
 13   first_name:Andy     

 76   first_name:Adam        -> Adam      prob:0.99479   
 77   first_name:Adam        -> Adam      prob:0.99828   
 78   first_name:Adam        -> Adam      prob:0.99413   
 79   first_name:Adam        -> Adam      prob:0.99401   
 80   first_name:Andrea      -> Andrea    prob:0.96524   
 81   first_name:Amir        -> Amir      prob:0.96967   
 82   first_name:Anteneh     -> Anteneh   prob:0.91317   
-> prob <= 0.80. Re-trying with gpt-4-0125-preview
 82   first_name:Anteneh     -> Anteneh   prob:1.0   
 83   first_name:Aaron       -> Aaron     prob:0.99064   
 84   first_name:Andrea      -> Andrea    prob:0.75756   
-> prob <= 0.80. Re-trying with gpt-4-0125-preview
 84   first_name:Andrea      -> Andrea    prob:1.0   
 85   first_name:Adolfo      -> Adolfo    prob:0.99146   
 86   first_name:Adriel      -> Adriel    prob:0.99599   
 87   first_name:Alex        -> Alex      prob:0.96534   
 88   first_name:Anny        -> Anny      prob:0.7923   
-> prob <= 0.80. Re-trying with gpt-4

In [899]:
for item in response.choices[0].logprobs.content[0].top_logprobs:
    print(item, '   prob:', np.exp(item.logprob))

TopLogprob(token='Andrew', bytes=[65, 110, 100, 114, 101, 119], logprob=-0.31562153)    prob: 0.7293354295193871
TopLogprob(token='Andy', bytes=[65, 110, 100, 121], logprob=-1.3611232)    prob: 0.2563726574071094
TopLogprob(token='Preferred', bytes=[80, 114, 101, 102, 101, 114, 114, 101, 100], logprob=-4.3295755)    prob: 0.013173138300554861
TopLogprob(token='Tony', bytes=[84, 111, 110, 121], logprob=-8.034284)    prob: 0.0003241565434692151
TopLogprob(token='AJ', bytes=[65, 74], logprob=-8.703351)    prob: 0.00016602851620392184


In [900]:
row['Bio'], row['Email']

('Andrew Buchholtz is a New Haven business broker of the well-known  Woodbridge International business brokerage company who helps people sell their business in New Haven, CT and nearest regions. Andrew J. Buchholtz has served as a banker, consultant and operator in the media industry for nearly 25 years. Originally a consultant to magazine publishers and direct marketers at J.S. Eliezer Associates in New York City, he joined Veronis Suhler Stevenson in 1995 and spent ten years buying and selling media properties as a mergers &amp; acquisitions advisor.Included among the dozens of transactions he has executed, Andrew sold Fast Company Magazine for the Zuckerman Group to Gruner + Jahr (USA), sold White Directory Publishers to the Hearst Corporation, sold Clipper Magazine to Ganett, Inc. and sold Highline Media to Summit Business Media. Sectors in which he has completed assignments include marketing agencies, digital video content, tradeshows, stock photography, education, b2b media, loc

In [843]:
df[:20]

,Preferred Name,likelihood,First Name,Last Name,Email,Company Name,Bio,Address,Country,State,City,Zip Code,Phone #,Phone #.1,Website,Profile Urls
0,Chris,1.00,Christopher,Cheung,chris.cheung@fbxbrokers.com,Florida Business Exchange,Chris Cheung is a \t\t\t\t\t\tTampa business b...,550 Reo St.\t\t\t\t\t\t\t\t\t\t\t\t\tSuite 300,United States,FL,Tampa,33602,813-785-9089,NaN,https://fbxbrokers.com,https://brokers.interexo.com/united-states_flo...
1,Bill,0.99,William,Gaines,bill@franpro.com,Franprocom,William Gaines is a business broker of the we...,Bowie,United States,MD,20715,NaN,443-583-3070,NaN,https://franpro.com,https://brokers.interexo.com/united-states_mar...
2,Tony,1.00,Anthony,Shard,tony@shardmb.com,Shard Marketing,Anthony Shard is a \t\t\t\t\t\tCortez business...,12325 Baypointe Ter,United States,FL,Cortez,34215-2566,810-577-5291,NaN,https://shardmb.com,https://brokers.interexo.com/united-states_flo...
3,Tony,1.00,Antonio,Bardy,tonybardy@gmail.com,Florida Connexion Properties,Antonio Bardy is a \t\t\t\t\t\tWindermere busi...,1961 Westhill Run,United States,FL,Windermere,34786-6230,407-535-3639,NaN,NaN,https://brokers.interexo.com/united-states_flo...
4,Liz,0.99,Elizabeth,Berger,liz@lizberger.com,Berger Realty Group,Elizabeth Berger is a Coral Springs business b...,1955 University Dr,United States,FL,Coral Springs,33071,954-344-8222,954-650-7848,https://lizberger.com/,https://brokers.interexo.com/united-states_flo...
5,Eric,1.00,Eric,Nur,10@nur-realty.com,First Brickell,Eric Nur is a business broker of the well-kno...,NaN,United States,FL,NaN,NaN,561-702-3795,NaN,https://nur-realty.com,https://brokers.interexo.com/united-states_flo...
6,Edward,1.00,Edward,Lavine,4strategic-search@cox.net,Strategic Resource,Edward Lavine is a Coto De Caza business broke...,31762 Via Perdiz,United States,CA,Coto De Caza,92679,949-709-5200,NaN,https://cox.net,https://brokers.interexo.com/united-states_cal...
7,Andrea,0.66,Andrea,Atchison,a.atchison@sunbeltnetwork.com,Sunbelt - Riverside County,Andrea started her career as a teacher. She re...,4181 Flat Rock Drive,United States,CA,Riverside,92505,951-678-5671,NaN,https://sunbeltnetwork.com,https://brokers.interexo.com/united-states_cal...
8,Adam,0.99,Adam,Bradshaw,a.bradshaw@nerycorp.com,The Nery Corporation,Adam Bradshaw is a New Bedford business broker...,700 Pleasant St Ste 330,United States,MA,New Bedford,02740-6254,508-990-9800,NaN,https://nerycorp.com,https://brokers.interexo.com/united-states_mas...
9,Allan,0.99,Allan,Carmody,a.carmody@murphybusiness.com,Murphy Business&Financial Corporation,Allan Carmody is a Lake Oswego business broker...,4000 Kruse Way Place Suite 2-245,United States,OR,Lake Oswego,97035,503-675-6272,503-840-5759,https://murphybusiness.com/,https://brokers.interexo.com/united-states_ore...
